In [4]:
import numpy as np
import pandas as pd

# Lese die Excel-Datei mit dem Profil, welches variiert werden soll ein
df_profil = pd.read_excel("profil_winter.xlsx")

# Anzahl der Zufallsvariablen pro Zeitpunkt
num_variables = 32

# Anzahl der Szenarien
num_scenarios = 100

# Standardabweichung der normalverteilten Zufallsvariablen
std_dev = 0.005

# Erstelle ein leeres Pandas-DataFrame für die Ergebnisse
results = pd.DataFrame()

# Für jedes Szenario:   
for i in range(num_scenarios):
    # Erstelle ein Array mit normalverteilten Zufallsvariablen für jeden Bus und jeden Zeitpunkt
    random_variables = np.random.normal(0, std_dev, (len(df_profil), num_variables))
    
    # Berechne die Summe der Zufallsvariablen für jeden Zeitpunkt
    sum_random_variables = np.sum(random_variables, axis=1)
       
    # Berechne den Faktor, um den jede Zufallsvariable angepasst werden muss, damit die Summe 0 ergibt
    factor = -sum_random_variables / num_variables

    # Passe jede Zufallsvariable an, indem der Faktor multipliziert wird
    random_variables_adjusted = random_variables + factor.reshape(-1, 1)

    # Die gewünschte Toleranz weil die Berechnung von Gleitkommazahlen ungenau ist und somit die Summe der angepassten Zufallszahlen minimal von 0 abweichen kann
    tolerance = 1e-10

    # Prüfe, ob die Summe der angepassten Zufallsvariablen für jeden Zeitpunkt innerhalb der Toleranz ist
    assert np.allclose(np.sum(random_variables_adjusted, axis=1), 0, atol=tolerance)
    
    # Füge die Zufallsvariablen ein
    df_varied = pd.DataFrame(random_variables_adjusted)

    # Füge eine Spalte für den Zeitpunkt hinzu
    df_varied.insert(0, "t", range(1, len(df_profil)+1))

    # Setze die Spaltenüberschriften auf "S" + Nummer des Szenarios
    #df_varied.columns = ["S{}".format(i+1) for i in range(num_variables)]

    # Füge das DataFrame mit den angepassten Zufallsvariablen dem DataFrame der Ergebnisse hinzu
    results = pd.concat([results, df_varied], ignore_index=True)

# Füge eine Spalte für die Szenarien hinzu
results.insert(0, "Scenario", np.repeat(np.arange(1, num_scenarios+1), len(df_profil)))

# Sortiere die Ergebnisse nach Szenario und Zeitpunkt
results = results.sort_values(by=["Scenario", "t"])

# Speichere das DataFrame der Ergebnisse in einer CSV-Datei
results.to_excel("Ergebnisse.xlsx", index=False)
